In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
from datatorch.agent.flows import Flow
import os
import logging

logging.basicConfig(
    format="%(levelname)-8s %(message)s",
    level=logging.DEBUG,
)

In [8]:
flow = Flow.from_yaml('flow.yaml')
action = flow.jobs[0].steps[0].action()
action.run(None, None)
action = flow.jobs[0].steps[1].action()
action.run(None, None)
action = flow.jobs[0].steps[2].action()
action.run(None, None)

DEBUG    Action found locally (datatorch/example-command@1).
INFO     Running command 'echo hello world'
DEBUG    Action found locally (datatorch/example-commands@1).
INFO     Running command 'echo line 1
echo line 2
echo line 3
echo line 4
echo line 5
'
DEBUG    Action found locally (datatorch/example-shell@1).
INFO     Running shell script '/home/justin/datatorch/actions/datatorch/example-shell/1/echo.sh'


SUPER
hello world

SUPER
line 1
line 2
line 3
line 4
line 5

SUPER
Hello world 1
Hello world 2
Hello world 3
Hello world 4
Hello world 5



In [ ]:
action = flow.jobs[0].steps[1].action()
action.run(None, None)